# Comparação entre o Crescimento Vegetativo e a Decomposição do MKT Share

# Setup Inicial

## Bibliotecas utilizadas

In [1]:
import pandas as pd
import numpy as np
import json

# Manipulação de datas
from time import strftime
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Visualização
import matplotlib.pyplot as plt
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objs import *

## Definições Gerais

In [2]:
# definção dos diretórios relativos

# replace with your container name
blob_container_name = 'general' 

# replace with your relative folder path
blob_relative_path_raw = 'raw/mercado_potencial/'
blob_relative_path_enriched = 'enriched/mercado_potencial/'


# replace with your linked service name
linked_service_raw = 'LS_ADLS_RAW_01'
linked_service_enriched = 'LS_ADLS_ENRICHED_01'

ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2]


dir_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
dir_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)


In [3]:
# dir_row  = 'abfss://general@stedlk01dtandev.dfs.core.windows.net/raw/mercado_potencial/'
# dir_enriched = 'abfss://general@stedlk02dtandev.dfs.core.windows.net/enriched/mercado_potencial/'

# ls_row = 'LS_ADLS_ROW_01'
# ls_enriched = 'LS_ADLS_ENRICHED_01'

## Funções auxiliares

In [4]:
def delta_indice(df_ind, df_vol):
    """ Criação do delta dos índice de volume de vendas diesel
        
        Cria serie de deltas, baseados nas variações do índice de volume de vendas a ser 
        aplicado no último mês de volume de dados disponível nas séries de dados.
    
    Parâmetros:
    -------------
    df_ind: pd.DataFrame
        Dataframe contendo os indices que serão consumidos
    df_vol: pd.DataFrame
        Dataframe com os volumes do MKTshare
    Retornos:
    -------------
    df2: pd.DataFrame
        Dataframe com o Delta a ser aplicado nos volumes
    start: datatime
        Data de início que será aplicado o Delta. A partir do primeiro mês projetado.
    """

    pri = df_ind.loc[pd.isna(df_ind['y_Indice_ConsumidorFinal']), :].index[0] #primeiro nan
    start_i = df_ind.loc[pri,'ds'] - relativedelta(months=1) # inicio do periodo a ser projetado
    start_v = df_vol.index.max()
    start = start_v.strftime('%Y-%m-%d') if start_v <= start_i else start_i.strftime('%Y-%m-%d')
    
    df2 = df_ind[df_ind['ds']>=start][['ds','y_Indice_ConsumidorFinal','yhat']] # filtro a partir do inicio do periodo
    df2.reset_index(inplace=True, drop=True)
    df2.columns = ['ds','y', 'yhat']
    df2['Delta'] = df2['yhat'].div(df2['yhat'][0], axis=0) # delta com base no último mês de indice do volume disponível

    return(df2, start)

# Coleta dos Volumes de Vendas por Atividade Econômica

## GET Histórico Vendas VIBRA por Atividades Econômicas

In [5]:
cf  = pd.read_parquet(dir_enriched + 'volumetria/vol01_vendas_vibra_mes.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})

## GET Volume Vendas Consumidor Final Vibra MKTShare Decomposto por Atividade Econômica

In [6]:
vol  = pd.read_parquet(dir_enriched + 'volumetria/vol02_MKTS_ConsFinal_Decomposto.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})

# Projeções

## Volume do MKTShare Decomposto do Consumidor Final para Transporte de Carga e de Passageiros

In [7]:
# VOLUME DECOMPOSTO DO MKTSHARE
#vol[['TRANSPORTE DE CARGAS','TRANSPORTE DE PASSAGEIRO']]

## Volume Histórico de Vendas Vibra para Transporte de Carga e de Passageiros

In [8]:
# Volume histórico Vibra
#cf[['TRANSPORTE DE CARGAS','TRANSPORTE DE PASSAGEIRO']]

## Resgatando os Índices de Transporte de Carga e de Passageiros dos modelos de Consumidor Final

In [9]:
indice_cg  = pd.read_parquet(dir_enriched + 'output_models/previsao_ConsFinal_Carga.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})
indice_ps  = pd.read_parquet(dir_enriched + 'output_models/previsao_ConsFinal_Passageiros.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})

## Criação do Delta do índice de volume de venda previsto a partir do último mês de dados reais

In [10]:
delta_carga, start = delta_indice(indice_cg, vol)
delta_passag, start = delta_indice(indice_ps, vol)

## Aplicando os Deltas

### Aos Volumes de Transporte de Cargas

In [11]:
# vol MKT Share
vol1 = vol[['TRANSPORTE DE CARGAS','TRANSPORTE DE PASSAG']]
vol1 = vol1[start:start]
x1 = delta_carga['Delta'].mul(vol1['TRANSPORTE DE CARGAS'].values[0],axis=0)

# volume histórico vibra
cf1 = cf[['TRANSPORTE DE CARGAS','TRANSPORTE DE PASSAG']]
cf1 = cf1[start:start]
x2 = delta_carga['Delta'].mul(cf1['TRANSPORTE DE CARGAS'].values[0],axis=0)

In [12]:
proj_cg = pd.concat([delta_carga, x1,x2,], ignore_index=True, axis=1)
proj_cg.columns = ['data', 'y', 'indice_carga', 'delta','proj_mktsh','proj_veget' ]
proj_cg = proj_cg.drop(['y'], axis=1)

proj_cg.insert(1,
               'data_str', 
               proj_cg['data'].dt.strftime('%Y-%m-%d').astype('string'),
               True) # auxilio para leitura da data em formato string pelo tableau
#proj_cg

### Aos Volumes de Transporte de Passageiros

In [13]:
# vol MKT Share
vol1 = vol[['TRANSPORTE DE CARGAS','TRANSPORTE DE PASSAG']]
vol1 = vol1[start:start]
x1 = delta_passag['Delta'].mul(vol1['TRANSPORTE DE PASSAG'].values[0],axis=0)

# volume histórico vibra
cf1 = cf[['TRANSPORTE DE CARGAS','TRANSPORTE DE PASSAG']]
cf1 = cf1[start:start]
x2 = delta_passag['Delta'].mul(cf1['TRANSPORTE DE PASSAG'].values[0],axis=0)


In [14]:
proj_ps = pd.concat([delta_passag, x1,x2,], ignore_index=True, axis=1)
proj_ps.columns = ['data', 'y', 'indice_passageiros', 'delta','proj_mktsh','proj_veget' ]
proj_ps = proj_ps.drop(['y'], axis=1)

proj_ps.insert(1,
               'data_str', 
               proj_ps['data'].dt.strftime('%Y-%m-%d').astype('string'),
               True) # auxilio para leitura da data em formato string pelo tableau
#proj_ps

## Salvando as Projeções

In [15]:
proj_cg.to_parquet(dir_enriched + 'volumetria/vol03_projecao_carga.parquet', 
                   storage_options = {'linked_service':linked_service_enriched})
                   
proj_ps.to_parquet(dir_enriched + 'volumetria/vol03_projecao_passageiro.parquet',
                   storage_options = {'linked_service':linked_service_enriched})

## Resgatando previsão vegetativa dos Transporte de Carga e de Passageiros

In [16]:
prev_vibra  = pd.read_parquet(dir_enriched + 'volumetria/vol01_prev_vegetativo.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})

In [17]:
prev_cg = prev_vibra[prev_vibra['Atv_Economica']=='TRANSPORTE DE CARGAS']
prev_ps = prev_vibra[prev_vibra['Atv_Economica']=='TRANSPORTE DE PASSAGEIRO']

In [18]:
end = proj_cg['data'].max()#+ relativedelta(months=1)
end = end.strftime('%Y-%m-%d')
x3 = prev_cg[(prev_cg['ds']>=start) & (prev_cg['ds']<=end)]['yhat'].reset_index(drop=True)
x4 = prev_ps[(prev_ps['ds']>=start) & (prev_ps['ds']<=end)]['yhat'].reset_index(drop=True)

## Comparativos

In [19]:
# remover a data em formato string
proj_cg.drop(['data_str'],axis=1, inplace=True)

comparativo_cg = pd.concat([proj_cg, x3], axis=1)
comparativo_cg.rename(columns={'yhat':'prev_veget'}, inplace=True)
#comparativo_cg

In [20]:
# remover a data em formato string
proj_ps.drop(['data_str'],axis=1, inplace=True)

comparativo_ps = pd.concat([proj_ps, x4], axis=1)
comparativo_ps.rename(columns={'yhat':'prev_veget'}, inplace=True)
#comparativo_ps

In [21]:
# comparativo_cg.to_csv(dir_enriched + 'volumetria/comparativo_cg.csv', index=False, sep=';', decimal=',',
#                       storage_options = {'linked_service':ls_enriched})

# comparativo_ps.to_csv(dir_enriched + 'volumetria/comparativo_ps.csv', index=False, sep=';', decimal=',',
#                       storage_options = {'linked_service':ls_enriched})

In [22]:
cores = ['red','orange','green', 'purple', 'blue', 'red']
series= ['Proj Mkt Share','Proj Decomposição Vegetativo', 'Previsão Vegetativo']

# fig = go.Figure()

fig = make_subplots(rows=2, cols=1,
                        y_title='Volume (m³)',
                        subplot_titles=('Transporte de Cargas', 'Transporte de Passageiros'),
                        shared_yaxes=False,
                        vertical_spacing=0.1 )

for i, c in enumerate(comparativo_cg.columns[3:]):
# linhas do Consumidor Final
        dt = comparativo_cg
        fig.add_trace(go.Scatter(
                        name = series[i],
                        x = dt['data'],
                        y = dt.iloc[:,3+i],
                        mode='lines+markers',
                        marker={'color': cores[i],
                                'size': 4,
                                'line': {'color': cores[i],
                                        'width': .65}
                                },
                        visible=True),
                    row=1, col=1
        )
# linhas do TRR
        dt = comparativo_ps
        fig.add_trace(go.Scatter(
                        name = series[i],
                        x = dt['data'],
                        y = dt.iloc[:,3+i],
                        mode='lines+markers',
                        marker={'color': cores[i],
                                'size': 4,
                                'line': {'color': cores[i],
                                        'width': .65}
                                },
                        visible=True,
                        showlegend=False),
                    row=2, col=1
        )
        
fig.add_annotation(
                showarrow=False,
                text='Gerado em ' + datetime.now().strftime("%d %b %Y às %H:%M:%S"),
                font=dict(size=12),
                xref='paper',
                x=1.1,
                yref='paper',
                y=-0.25)
    
fig.update_xaxes(matches='x', rangeslider_visible=True, rangeslider_thickness = 0.08)
fig.update_xaxes(rangeslider= {'visible':False}, row=1, col=1)

ano_s = dt['data'].min().strftime('%m.%Y')
ano_e = dt['data'].max().strftime('%m.%Y')

fig.update_layout(
                title = 'Comparativo Volumes Previstos e Projetados<br><sup>('+ ano_s +' - '+ ano_e+ ')</sup>',
                #xaxis_title = "Período",
                #yaxis_title = "% mercado",
                legend_title = "Legenda",
                font=dict(
                    family = "Courier New, monospace",
                    size = 14,
                    color = "royalblue"),
                    height=900)
#
fig.show()
fig = plotly.offline.plot(fig, output_type='div')
#displayHTML(fig)
        

#

# Acesso à Documentação

In [23]:
help(delta_indice)